In [16]:
import requests
import pandas as pd
from pathlib import Path
import time

BASE_URL = "https://api.openf1.org/v1"
RAW_DATA_DIR = Path("data/raw")

ENDPOINTS = {
    # "overtakes": "/overtakes",
    # "stints": "/stints",
    # "pit": "/pit",
    # "session_result": "/session_result",
    # "starting_grid": "/starting_grid",
    # "race_control": "/race_control",
    # "weather": "/weather",
    # "position": "/position",
    # "laps": "/laps",
    #"championship_drivers": "/championship_drivers", 
    "drivers": "/drivers"
}

In [17]:
def fetch_endpoint(endpoint: str, params: dict) -> list:
    """
    Fetch data from OpenF1 API endpoint.
    Returns JSON (list of dicts).
    """
    url = f"{BASE_URL}{endpoint}"
    response = requests.get(url, params=params, timeout=30)
    response.raise_for_status()
    return response.json()

def save_csv(data: list, season: int, race_name: str, filename: str):
    """
    Save list-of-dicts data as CSV in a structured folder.
    """
    if not data:
        print(f"[WARN] Empty data for {filename}")
        return

    path = RAW_DATA_DIR / f"{season}" / f"{race_name}"
    path.mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data)
    file_path = path / f"{filename}.csv"
    df.to_csv(file_path, index=False)

    print(f"[OK] Saved {file_path} ({len(df)} rows)")

In [4]:
year = 2025
data = fetch_endpoint(
    endpoint="/meetings",
    params={
        "year": year
    }
)
df = pd.DataFrame(data)
df.to_csv(f'data/raw/{year}/meetings.csv', index=False)

In [18]:
year = 2025
data = fetch_endpoint(
    endpoint="/sessions",
    params={
        "year": year,
        "session_type": "Race"
    }
)
df = pd.DataFrame(data)
df.to_csv(f'data/raw/{year}/race_sesions.csv', index=False)

In [19]:
SEASON = 2025
for row in df.itertuples(index=True):
    RACE_NAME = f'{row.country_code} - {row.circuit_short_name}'
    SESSION_KEY = row.session_key

    for name, endpoint in ENDPOINTS.items():
        print(f"Fetching {name}...")
        
        data = fetch_endpoint(
            endpoint=endpoint,
            params={"session_key": SESSION_KEY}
        )
        
        save_csv(
            data=data,
            season=SEASON,
            race_name=RACE_NAME,
            filename=name,
        )
        time.sleep(2)


Fetching drivers...
[OK] Saved data/raw/2025/AUS - Melbourne/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/CHN - Shanghai/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/CHN - Shanghai/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/JPN - Suzuka/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/BRN - Sakhir/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/KSA - Jeddah/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/USA - Miami/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/USA - Miami/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/ITA - Imola/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/MON - Monte Carlo/drivers.csv (20 rows)
Fetching drivers...
[OK] Saved data/raw/2025/ESP - Catalunya/drivers.csv (19 rows)
Fetching drivers...
[OK] Saved data/raw/2025/CAN - Montreal/drivers.csv (20 rows)
Fetching drivers...
[OK] S

In [11]:
RACE_NAME = f'BRA - Interlagos'
SESSION_KEY = 9869

for name, endpoint in ENDPOINTS.items():
    print(f"Fetching {name}...")
    
    data = fetch_endpoint(
        endpoint=endpoint,
        params={"session_key": SESSION_KEY}
    )
    
    save_csv(
        data=data,
        season=SEASON,
        race_name=RACE_NAME,
        filename=name,
    )
    time.sleep(2)

Fetching overtakes...
[OK] Saved data/raw/2025/BRA - Interlagos/overtakes.csv (369 rows)
Fetching stints...
[OK] Saved data/raw/2025/BRA - Interlagos/stints.csv (57 rows)
Fetching pit...
[OK] Saved data/raw/2025/BRA - Interlagos/pit.csv (37 rows)
Fetching session_result...
[OK] Saved data/raw/2025/BRA - Interlagos/session_result.csv (20 rows)
Fetching starting_grid...
[WARN] Empty data for starting_grid
Fetching race_control...
[OK] Saved data/raw/2025/BRA - Interlagos/race_control.csv (58 rows)
Fetching weather...
[OK] Saved data/raw/2025/BRA - Interlagos/weather.csv (152 rows)
Fetching position...
[OK] Saved data/raw/2025/BRA - Interlagos/position.csv (759 rows)
Fetching laps...
[OK] Saved data/raw/2025/BRA - Interlagos/laps.csv (1252 rows)


In [25]:
import fastf1
from fastf1 import plotting
import pandas as pd



def get_race_lap_data(year: int, gp_round: int):
    # Загружаем сессию гонки
    session = fastf1.get_session(year, gp_round, 'R')
    session.load()

    # Получаем таблицу laps (lap-by-lap)
    lap_data = session.laps


    # Преобразуем время в секунды (удобно для анализа)
    def to_seconds(td):
        return td.total_seconds() if pd.notnull(td) else None

    lap_data['LapTimeSec'] = lap_data['LapTime'].apply(to_seconds)
    lap_data['S1_sec'] = lap_data['Sector1Time'].apply(to_seconds)
    lap_data['S2_sec'] = lap_data['Sector2Time'].apply(to_seconds)
    lap_data['S3_sec'] = lap_data['Sector3Time'].apply(to_seconds)

    return lap_data

In [26]:
df = get_race_lap_data(year=2025, gp_round=6)

df

core           INFO 	Loading data for Miami Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '6'
core        WARNING 	Fixed incorrect tyre stint information for driver '31'
core        WARNING 	Fixed incorrect tyre stint information for driver '18'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,LapTimeSec,S1_sec,S2_sec,S3_sec
0,0 days 00:58:51.257000,VER,1,0 days 00:01:35.669000,1.0,NaN,NaT,NaT,NaT,0 days 00:00:34.681000,...,12,1.0,False,,False,False,95.669,NaN,34.681,26.310
1,0 days 01:00:42.385000,VER,1,0 days 00:01:51.128000,2.0,NaN,NaT,NaT,0 days 00:00:31.426000,0 days 00:00:47.940000,...,126,1.0,False,,False,False,111.128,31.426,47.940,31.762
2,0 days 01:02:48.331000,VER,1,0 days 00:02:05.946000,3.0,NaN,NaT,NaT,0 days 00:00:43.901000,0 days 00:00:48.777000,...,671,1.0,False,,False,False,125.946,43.901,48.777,33.268
3,0 days 01:04:20.900000,VER,1,0 days 00:01:32.569000,4.0,NaN,NaT,NaT,0 days 00:00:31.185000,0 days 00:00:35.126000,...,1,1.0,False,,False,True,92.569,31.185,35.126,26.258
4,0 days 01:05:53.033000,VER,1,0 days 00:01:32.133000,5.0,NaN,NaT,NaT,0 days 00:00:31.182000,0 days 00:00:34.799000,...,1,1.0,False,,False,True,92.133,31.182,34.799,26.152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,0 days 01:35:45.563000,BEA,87,0 days 00:01:32.680000,24.0,1.0,NaT,NaT,0 days 00:00:31.083000,0 days 00:00:35.480000,...,1,12.0,False,,False,True,92.680,31.083,35.480,26.117
1001,0 days 01:37:18.617000,BEA,87,0 days 00:01:33.054000,25.0,1.0,NaT,NaT,0 days 00:00:31.073000,0 days 00:00:35.848000,...,1,12.0,False,,False,True,93.054,31.073,35.848,26.133
1002,0 days 01:38:52.757000,BEA,87,0 days 00:01:34.140000,26.0,1.0,NaT,NaT,0 days 00:00:32.323000,0 days 00:00:35.617000,...,1,12.0,False,,False,True,94.140,32.323,35.617,26.200
1003,0 days 01:40:26.361000,BEA,87,0 days 00:01:33.604000,27.0,1.0,NaT,NaT,0 days 00:00:31.130000,0 days 00:00:35.590000,...,1,13.0,False,,False,True,93.604,31.130,35.590,26.884


In [28]:
import fastf1
import pandas as pd
import os



def get_race_lap_data(year: int, round_: int):
    session = fastf1.get_session(year, round_, 'R')
    session.load()

    laps = session.laps

    # Если гонки нет или пусто — возвращаем None
    if laps.empty:
        return None, None

    lap_df = laps.copy()

    # Функция для конвертации времени в секунды
    def to_sec(td):
        return td.total_seconds() if pd.notnull(td) else None

    lap_df['LapTimeSec'] = lap_df['LapTime'].apply(to_sec)
    lap_df['S1_sec'] = lap_df['Sector1Time'].apply(to_sec)
    lap_df['S2_sec'] = lap_df['Sector2Time'].apply(to_sec)
    lap_df['S3_sec'] = lap_df['Sector3Time'].apply(to_sec)

    # Добавить название гонки
    race_name = session.event['EventName']

    return lap_df, race_name


def collect_season(year: int):
    print(f"Собираем сезон {year} …")

    # Список всех сессий гонок в сезоне
    schedule = fastf1.get_event_schedule(year)
    races_only = schedule[schedule["Session5"] == "Race"]

    print(f"Гонок в сезоне {year}: {len(races_only)}")
    season_data = []

    for idx, race in races_only.iterrows():
        round_number = race['RoundNumber']
        race_name = race['EventName']

        print(f" → Загружаем {race_name} (Round {round_number})")

        lap_df, race_label = get_race_lap_data(year, round_number)

        if lap_df is None:
            print(f"  ⚠ Пропущено: {race_name} — нет данных")
            continue

        # Добавляем колонку с именем гонки
        lap_df['Race'] = race_label

        # Сохраняем в локальный CSV
        fname = f"{year}_{round_number:02d}_{race_label.replace(' ', '_')}.csv"
        lap_df.to_csv(fname, index=False)
        print(f"   ✔ Сохранено: {fname}")

        season_data.append(lap_df)

    # Собираем общий DataFrame
    if season_data:
        all_season_df = pd.concat(season_data, ignore_index=True)
        all_fname = f"{year}_season_all_races.csv"
        all_season_df.to_csv(all_fname, index=False)
        print(f"\nОбщий файл сезона: {all_fname}")

    print("Готово!")


# Запуск
collect_season(2025)

core           INFO 	Loading data for Australian Grand Prix - Race [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...


Собираем сезон 2025 …
Гонок в сезоне 2025: 24
 → Загружаем Australian Grand Prix (Round 1)


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

   ✔ Сохранено: 2025_01_Australian_Grand_Prix.csv
 → Загружаем Chinese Grand Prix (Round 2)


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written t

...

   ✔ Сохранено: 2025_22_Las_Vegas_Grand_Prix.csv
 → Загружаем Qatar Grand Prix (Round 23)


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

   ✔ Сохранено: 2025_23_Qatar_Grand_Prix.csv
 → Загружаем Abu Dhabi Grand Prix (Round 24)


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written t

   ✔ Сохранено: 2025_24_Abu_Dhabi_Grand_Prix.csv

Общий файл сезона: 2025_season_all_races.csv
Готово!
